In [1]:


# 2016년에 가장 관심을 많이 받았던 비감독(Unsupervised) 학습 방법인
# Generative Adversarial Network(GAN)을 구현
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import image
img_data = image.load_img('./cat_photos/chrtreux31.jpg', target_size=(64,64))


Using TensorFlow backend.


In [2]:
pix =np.array(img_data)
pix =pix.astype('float32')
pix /= 255.0
print(pix.shape)
pix = pix.reshape(-1,12288)
pix

(64, 64, 3)


array([[0.77254903, 0.78039217, 0.8627451 , ..., 0.13333334, 0.14509805,
        0.17254902]], dtype=float32)

In [3]:
#########
# 옵션 설정
#########
total_epoch = 1000
batch_size = 1
learning_rate = 0.0001
# 신경망 레이어 구성 옵션
n_hidden = 256
n_input = 12288
n_noise = 64  # 생성기의 입력값으로 사용할 노이즈의 크기

#########
# 신경망 모델 구성
######
# GAN 도 Unsupervised 학습이므로 Autoencoder 처럼 Y 를 사용하지 않습니다.
X = tf.placeholder(tf.float32, [None, n_input])
# 노이즈 Z를 입력값으로 사용합니다.
Z = tf.placeholder(tf.float32, [None, n_noise])

# 생성기 신경망에 사용하는 변수들입니다.
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 판별기 신경망에 사용하는 변수들입니다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
# 판별기의 최종 결과값은 얼마나 진짜와 가깝냐를 판단하는 한 개의 스칼라값입니다.
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))


# 생성기(G) 신경망을 구성합니다.
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)

    return output


# 판별기(D) 신경망을 구성합니다.
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)

    return output


# 랜덤한 노이즈(Z)를 만듭니다.
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))


# 노이즈를 이용해 랜덤한 이미지를 생성합니다.
G = generator(Z)
# 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구합니다.
D_gene = discriminator(G)
# 진짜 이미지를 이용해 판별한 값을 구합니다.
D_real = discriminator(X)

# 논문에 따르면, GAN 모델의 최적화는 loss_G 와 loss_D 를 최대화 하는 것 입니다.
# 다만 loss_D와 loss_G는 서로 연관관계가 있기 때문에 두 개의 손실값이 항상 같이 증가하는 경향을 보이지는 않을 것 입니다.
# loss_D가 증가하려면 loss_G는 하락해야하고, loss_G가 증가하려면 loss_D는 하락해야하는 경쟁관계에 있기 때문입니다.
# 논문의 수식에 따른 다음 로직을 보면 loss_D 를 최대화하기 위해서는 D_gene 값을 최소화하게 됩니다.
# 판별기에 진짜 이미지를 넣었을 때에도 최대값을 : tf.log(D_real)
# 가짜 이미지를 넣었을 때에도 최대값을 : tf.log(1 - D_gene)
# 갖도록 학습시키기 때문입니다.
# 이것은 판별기는 생성기가 만들어낸 이미지가 가짜라고 판단하도록 판별기 신경망을 학습시킵니다.
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
# 반면 loss_G 를 최대화하기 위해서는 D_gene 값을 최대화하게 되는데,
# 이것은 가짜 이미지를 넣었을 때, 판별기가 최대한 실제 이미지라고 판단하도록 생성기 신경망을 학습시킵니다.
# 논문에서는 loss_D 와 같은 수식으로 최소화 하는 생성기를 찾지만,
# 결국 D_gene 값을 최대화하는 것이므로 다음과 같이 사용할 수 있습니다.
loss_G = tf.reduce_mean(tf.log(D_gene))

# loss_D 를 구할 때는 판별기 신경망에 사용되는 변수만 사용하고,
# loss_G 를 구할 때는 생성기 신경망에 사용되는 변수만 사용하여 최적화를 합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# GAN 논문의 수식에 따르면 loss 를 극대화 해야하지만, minimize 하는 최적화 함수를 사용하기 때문에
# 최적화 하려는 loss_D 와 loss_G 에 음수 부호를 붙여줍니다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                         var_list=G_var_list)



In [4]:
#########
# 신경망 모델 학습
#########
sess = tf.Session()
sess.run(tf.global_variables_initializer())

loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    noise = get_noise(batch_size, n_noise)
    # 판별기와 생성기 신경망을 각각 학습시킵니다.
    _, loss_val_D = sess.run([train_D, loss_D],feed_dict={X: pix, Z: noise})
    _, loss_val_G = sess.run([train_G, loss_G],feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))
     #########
    # 학습이 되어가는 모습을 보기 위해 주기적으로 이미지를 생성하여 저장
    ######
    if epoch == 0 or (epoch + 1) % 100 == 0:
        sample_size = 1
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        samples = np.reshape(samples, (64, 64, 3))
        fig, ax = plt.subplots(1, sample_size, figsize=(10, 10))
        ax.set_axis_off()
        ax.imshow(samples)
        plt.savefig('samples/{}.png'.format(str(epoch+1).zfill(3)), bbox_inches='tight')
        plt.close(fig)


print('최적화 완료!')

Epoch: 0000 D loss: -1.401 G loss: -0.7157
Epoch: 0001 D loss: -1.244 G loss: -0.7832
Epoch: 0002 D loss: -1.121 G loss: -0.8653
Epoch: 0003 D loss: -1.008 G loss: -0.9487
Epoch: 0004 D loss: -0.9048 G loss: -1.033
Epoch: 0005 D loss: -0.8083 G loss: -1.116
Epoch: 0006 D loss: -0.7209 G loss: -1.202
Epoch: 0007 D loss: -0.6415 G loss: -1.294
Epoch: 0008 D loss: -0.5688 G loss: -1.397
Epoch: 0009 D loss: -0.5025 G loss: -1.508
Epoch: 0010 D loss: -0.4417 G loss: -1.634
Epoch: 0011 D loss: -0.3888 G loss: -1.755
Epoch: 0012 D loss: -0.3414 G loss: -1.875
Epoch: 0013 D loss: -0.2997 G loss: -1.995
Epoch: 0014 D loss: -0.2626 G loss: -2.111
Epoch: 0015 D loss: -0.2309 G loss: -2.222
Epoch: 0016 D loss: -0.202 G loss: -2.341
Epoch: 0017 D loss: -0.1777 G loss: -2.457
Epoch: 0018 D loss: -0.1574 G loss: -2.566
Epoch: 0019 D loss: -0.1392 G loss: -2.676
Epoch: 0020 D loss: -0.1238 G loss: -2.787
Epoch: 0021 D loss: -0.11 G loss: -2.9
Epoch: 0022 D loss: -0.09777 G loss: -3.012
Epoch: 0023 D l

Epoch: 0199 D loss: -0.004032 G loss: -6.034
Epoch: 0200 D loss: -0.00324 G loss: -6.409
Epoch: 0201 D loss: -0.004811 G loss: -5.75
Epoch: 0202 D loss: -0.003914 G loss: -6.07
Epoch: 0203 D loss: -0.003328 G loss: -6.348
Epoch: 0204 D loss: -0.005418 G loss: -5.591
Epoch: 0205 D loss: -0.003378 G loss: -6.339
Epoch: 0206 D loss: -0.002729 G loss: -6.777
Epoch: 0207 D loss: -0.006757 G loss: -5.314
Epoch: 0208 D loss: -0.00511 G loss: -5.714
Epoch: 0209 D loss: -0.002563 G loss: -7.044
Epoch: 0210 D loss: -0.003106 G loss: -6.599
Epoch: 0211 D loss: -0.00312 G loss: -6.594
Epoch: 0212 D loss: -0.002877 G loss: -6.778
Epoch: 0213 D loss: -0.002615 G loss: -7.009
Epoch: 0214 D loss: -0.003324 G loss: -6.415
Epoch: 0215 D loss: -0.003801 G loss: -6.148
Epoch: 0216 D loss: -0.008151 G loss: -5.077
Epoch: 0217 D loss: -0.002418 G loss: -7.177
Epoch: 0218 D loss: -0.002439 G loss: -7.153
Epoch: 0219 D loss: -0.002419 G loss: -7.155
Epoch: 0220 D loss: -0.003819 G loss: -6.125
Epoch: 0221 D l

Epoch: 0400 D loss: -0.1048 G loss: -2.269
Epoch: 0401 D loss: -0.08038 G loss: -2.668
Epoch: 0402 D loss: -0.01517 G loss: -4.343
Epoch: 0403 D loss: -0.001057 G loss: -7.196
Epoch: 0404 D loss: -0.03906 G loss: -3.447
Epoch: 0405 D loss: -0.003647 G loss: -5.873
Epoch: 0406 D loss: -0.0003518 G loss: -10.15
Epoch: 0407 D loss: -0.000503 G loss: -8.876
Epoch: 0408 D loss: -0.003824 G loss: -5.814
Epoch: 0409 D loss: -0.01136 G loss: -4.665
Epoch: 0410 D loss: -0.004471 G loss: -5.67
Epoch: 0411 D loss: -0.9092 G loss: -1.129
Epoch: 0412 D loss: -0.2193 G loss: -2.827
Epoch: 0413 D loss: -0.0128 G loss: -10.32
Epoch: 0414 D loss: -0.055 G loss: -11.42
Epoch: 0415 D loss: -0.1699 G loss: -7.68
Epoch: 0416 D loss: -0.2923 G loss: -7.757
Epoch: 0417 D loss: -2.811 G loss: -0.1756
Epoch: 0418 D loss: -0.5791 G loss: -5.584
Epoch: 0419 D loss: -0.4932 G loss: -9.478
Epoch: 0420 D loss: -0.1991 G loss: -7.157
Epoch: 0421 D loss: -0.05506 G loss: -8.081
Epoch: 0422 D loss: -0.02624 G loss: -3

Epoch: 0600 D loss: -0.4329 G loss: -4.411
Epoch: 0601 D loss: -0.3926 G loss: -5.339
Epoch: 0602 D loss: -0.3845 G loss: -2.012
Epoch: 0603 D loss: -0.9678 G loss: -0.5504
Epoch: 0604 D loss: -0.3089 G loss: -1.763
Epoch: 0605 D loss: -0.1214 G loss: -5.079
Epoch: 0606 D loss: -0.1056 G loss: -3.707
Epoch: 0607 D loss: -0.4262 G loss: -1.129
Epoch: 0608 D loss: -0.5772 G loss: -0.9585
Epoch: 0609 D loss: -0.1307 G loss: -2.756
Epoch: 0610 D loss: -0.917 G loss: -0.7081
Epoch: 0611 D loss: -0.4091 G loss: -1.601
Epoch: 0612 D loss: -0.1744 G loss: -3.846
Epoch: 0613 D loss: -0.6742 G loss: -1.172
Epoch: 0614 D loss: -0.9828 G loss: -0.8801
Epoch: 0615 D loss: -0.7538 G loss: -1.409
Epoch: 0616 D loss: -1.358 G loss: -0.6798
Epoch: 0617 D loss: -0.677 G loss: -2.949
Epoch: 0618 D loss: -0.5744 G loss: -3.849
Epoch: 0619 D loss: -0.9219 G loss: -0.6962
Epoch: 0620 D loss: -0.2618 G loss: -3.605
Epoch: 0621 D loss: -0.7495 G loss: -0.6705
Epoch: 0622 D loss: -1.0 G loss: -0.5384
Epoch: 06

Epoch: 0800 D loss: -1.163 G loss: -0.7714
Epoch: 0801 D loss: -1.053 G loss: -1.046
Epoch: 0802 D loss: -0.7249 G loss: -2.378
Epoch: 0803 D loss: -1.014 G loss: -0.9993
Epoch: 0804 D loss: -0.6337 G loss: -1.738
Epoch: 0805 D loss: -1.074 G loss: -0.5308
Epoch: 0806 D loss: -1.097 G loss: -0.5673
Epoch: 0807 D loss: -1.104 G loss: -0.6797
Epoch: 0808 D loss: -0.7827 G loss: -1.22
Epoch: 0809 D loss: -1.085 G loss: -0.9012
Epoch: 0810 D loss: -0.7758 G loss: -1.718
Epoch: 0811 D loss: -1.029 G loss: -0.961
Epoch: 0812 D loss: -0.922 G loss: -1.017
Epoch: 0813 D loss: -1.039 G loss: -0.7009
Epoch: 0814 D loss: -1.096 G loss: -0.6534
Epoch: 0815 D loss: -0.7708 G loss: -1.064
Epoch: 0816 D loss: -1.043 G loss: -0.7307
Epoch: 0817 D loss: -0.7657 G loss: -1.12
Epoch: 0818 D loss: -0.9467 G loss: -0.896
Epoch: 0819 D loss: -1.039 G loss: -0.8569
Epoch: 0820 D loss: -0.8106 G loss: -1.262
Epoch: 0821 D loss: -1.012 G loss: -0.88
Epoch: 0822 D loss: -0.8947 G loss: -1.005
Epoch: 0823 D loss

최적화 완료!
